# Importamos los modulos requeridos por la aplicación

In [1]:
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')
# importamos el modulo de sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Cargamos el dataset a utilizar

In [2]:
data = pd.read_csv('dataset_2.csv', index_col=0)
data.head()

,id,title,tags
0,862,toy story,tom hanks tim allen don rickles jim varney wal...
1,8844,jumanji,robin williams jonathan hyde kirsten dunst bra...
2,15602,grumpier old men,walter matthau jack lemmon ann-margret sophia ...
3,31357,waiting to exhale,whitney houston angela bassett loretta devine ...
4,11862,father of the bride part ii,steve martin diane keaton martin short kimberl...


# Hacemos el tratamiento de los datos que vamos a utilizar para la recomendacion

In [3]:
df = data[['title', 'tags']]
# df = df.sample(frac=0.1)
df = df.iloc[0:2001]
df.reset_index(drop=True, inplace=True)
df['tags'] = df['tags'].str.replace('[{}]'.format(string.punctuation), ' ')
df

,title,tags
0,toy story,tom hanks tim allen don rickles jim varney wal...
1,jumanji,robin williams jonathan hyde kirsten dunst bra...
2,grumpier old men,walter matthau jack lemmon ann margret sophia ...
3,waiting to exhale,whitney houston angela bassett loretta devine ...
4,father of the bride part ii,steve martin diane keaton martin short kimberl...
...,...,...
1996,l.a. story,steve martin sarah jessica parker victoria ten...
1997,the jerk,steve martin bernadette peters catlin adams ma...
1998,dead men don't wear plaid,steve martin rachel ward alan ladd carl reiner...
1999,the man with two brains,steve martin kathleen turner david warner paul...


Usamos el vectorizer de sklearn para calcular la frecuencia de las palabras que aparecen en nuestro tag y el parametro stop words para descartar todas las pablabras comunes del idioma ingles que no aportar valor a mi modelo

In [4]:
tfidf = TfidfVectorizer(stop_words='english')
df['tags'].fillna('', inplace=True) # reemplazamos los valores nulos del dataframe por un vacio
tfidf_matrix = tfidf.fit_transform(df['tags']) # creamos la matriz donde estaran las palabras de cada tag y su frecuencia
tfidf.vocabulary_ # imprimimos el vocabulario encontrado y su frecuencia a lo largo del dataframe

{'tom': 25361,
 'hanks': 10942,
 'tim': 25265,
 'allen': 802,
 'don': 7220,
 'rickles': 21221,
 'jim': 12924,
 'varney': 26342,
 'wallace': 26825,
 'shawn': 22830,
 'john': 12985,
 'ratzenberger': 20571,
 'annie': 1106,
 'potts': 19808,
 'morris': 17276,
 'erik': 8221,
 'von': 26707,
 'detten': 6734,
 'laurie': 14432,
 'metcalf': 16700,
 'lee': 14540,
 'ermey': 8228,
 'sarah': 22084,
 'freeman': 9387,
 'penn': 19153,
 'jillette': 12921,
 'animation': 1079,
 'comedy': 5141,
 'family': 8620,
 'led': 14531,
 'woody': 27440,
 'andy': 1039,
 'toys': 25521,
 'live': 14944,
 'happily': 10975,
 'room': 21543,
 'birthday': 2652,
 'brings': 3342,
 'buzz': 3711,
 'lightyear': 14801,
 'scene': 22235,
 'afraid': 533,
 'losing': 15131,
 'place': 19540,
 'heart': 11220,
 'plots': 19612,
 'circumstances': 4771,
 'separate': 22622,
 'owner': 18725,
 'duo': 7582,
 'eventually': 8359,
 'learns': 14506,
 'aside': 1462,
 'differences': 6873,
 'robin': 21388,
 'williams': 27264,
 'jonathan': 13019,
 'hyde':

In [5]:
# imprimimos el tamaño de la matrix
tfidf_matrix.shape

(2001, 27960)

# Implementamos la similaridad coseno

- calcula el nucleo lineal entre los parametros recibidos y me sirve para comparar al llamar una pelicula se compara con todas las otras peliculas y selecciona la que tiene mas relacion

In [6]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 

- Creamos una serie con el id de las peliculas

In [7]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# Creamos la funcion que hace la recomendacion

In [8]:
def recomendacion(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key= lambda x:x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movies_idx = [i[0] for i in sim_scores]
    return df['title'].iloc[movies_idx]

In [9]:
df.head()

,title,tags
0,toy story,tom hanks tim allen don rickles jim varney wal...
1,jumanji,robin williams jonathan hyde kirsten dunst bra...
2,grumpier old men,walter matthau jack lemmon ann margret sophia ...
3,waiting to exhale,whitney houston angela bassett loretta devine ...
4,father of the bride part ii,steve martin diane keaton martin short kimberl...


In [10]:
movie = df.title[120]
rec = recomendacion(title=movie)

# Imprimimos la recomendacion

In [11]:
print('La pelicula que viste es:', movie.title())
print('Tu recomendacion es: ')
for i, j  in enumerate(rec):
    print(str(i + 1) + ' ' + '-' + ' ' + j.title())

La pelicula que viste es: Boomerang
Tu recomendacion es: 
1 - Martin Lawrence: You So Crazy
2 - Picture Perfect
3 - Waiting To Exhale
4 - Harriet The Spy
5 - She'S So Lovely


# Verificamos que tanta coincidencia tienen la pelicula que vimos y la que nos recomiendo en primer lugar

In [12]:
file_csv = 'verificacion_movies.csv'
csv = pd.read_csv(file_csv, index_col=0)
csv.head()

,title,overview,name_actor,genero
0,toy story,"Led by Woody, Andy's toys live happily in his ...",Tom Hanks,Animation
1,toy story,"Led by Woody, Andy's toys live happily in his ...",Tim Allen,Animation
2,toy story,"Led by Woody, Andy's toys live happily in his ...",Don Rickles,Animation
3,toy story,"Led by Woody, Andy's toys live happily in his ...",Jim Varney,Animation
4,toy story,"Led by Woody, Andy's toys live happily in his ...",Wallace Shawn,Animation


In [13]:
data_ver = csv[['title', 'overview', 'name_actor', 'genero']]
data_ver.title = data_ver.title.str.lower()
data_ver.head()

,title,overview,name_actor,genero
0,toy story,"Led by Woody, Andy's toys live happily in his ...",Tom Hanks,Animation
1,toy story,"Led by Woody, Andy's toys live happily in his ...",Tim Allen,Animation
2,toy story,"Led by Woody, Andy's toys live happily in his ...",Don Rickles,Animation
3,toy story,"Led by Woody, Andy's toys live happily in his ...",Jim Varney,Animation
4,toy story,"Led by Woody, Andy's toys live happily in his ...",Wallace Shawn,Animation


# Comparamos los actores, overviews y los generos de la pelicula vista y la primera recomendacion

- Actores

In [14]:
data_ver = data_ver[(data_ver.title == movie) | (data_ver.title == rec.values[0])]
aux_act = data_ver[['title', 'name_actor']]
aux_act.drop_duplicates(inplace=True)
dict_list_actors = aux_act.to_dict('records')

nuevo_dicc_act = {}
for diccionario in dict_list_actors:
    clave = diccionario['title']
    valor = diccionario['name_actor']
    
    if clave in nuevo_dicc_act:
        nuevo_dicc_act[clave].append(valor)
    else:
        nuevo_dicc_act[clave] = [valor]

lista1 = nuevo_dicc_act[movie]
lista2 = nuevo_dicc_act[rec.values[0]]

elementos_comunes = set(lista1) & set(lista2)
cantidad_elementos_comunes = list(elementos_comunes)

print("Actores en comun comunes:", cantidad_elementos_comunes)

Actores en comun comunes: ['Martin Lawrence']


- Overviews

In [15]:
aux_ov = data_ver[['title', 'overview']]
aux_ov.drop_duplicates(inplace=True)
dict_list_gen = aux_ov.to_dict('records')

nuevo_dicc_ov = {}
for diccionario in dict_list_gen:
    clave = diccionario['title']
    valor = diccionario['overview']
    
    if clave in nuevo_dicc_ov:
        nuevo_dicc_ov[clave].append(valor)
    else:
        nuevo_dicc_ov[clave] = [valor]

lista3 = nuevo_dicc_ov[movie]
lista4 = nuevo_dicc_ov[rec.values[0]]

elementos_comunes = set(lista3) & set(lista4)
cantidad_elementos_comunes = list(elementos_comunes)

print("Palabras comunes en el overview de las peliculas vista y recomendada:", cantidad_elementos_comunes)

Palabras comunes en el overview de las peliculas vista y recomendada: []


- Generos


In [16]:
aux_gen = data_ver[['title', 'genero']]
aux_gen.drop_duplicates(inplace=True)
dict_list_gen = aux_gen.to_dict('records')

nuevo_dicc_gen = {}
for diccionario in dict_list_gen:
    clave = diccionario['title']
    valor = diccionario['genero']
    
    if clave in nuevo_dicc_gen:
        nuevo_dicc_gen[clave].append(valor)
    else:
        nuevo_dicc_gen[clave] = [valor]

lista5 = nuevo_dicc_gen[movie]
lista6 = nuevo_dicc_gen[rec.values[0]]

elementos_comunes = set(lista5) & set(lista6)
cantidad_elementos_comunes = list(elementos_comunes)

print("Generos en comun comunes:", cantidad_elementos_comunes)

Generos en comun comunes: ['Comedy']
